In [16]:
import numpy as np
import pandas as pd
pd.set_option("display.precision", 3)
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'

In [7]:
data = pd.read_csv('train_data.csv', index_col = 'ID')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59966 entries, 1 to 59966
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   HealthServiceArea             59966 non-null  object
 1   Gender                        59966 non-null  object
 2   Race                          59966 non-null  object
 3   TypeOfAdmission               59966 non-null  object
 4   CCSProcedureCode              59966 non-null  int64 
 5   APRSeverityOfIllnessCode      59966 non-null  int64 
 6   PaymentTypology               59966 non-null  object
 7   BirthWeight                   59966 non-null  int64 
 8   EmergencyDepartmentIndicator  59966 non-null  object
 9   AverageCostInCounty           59966 non-null  int64 
 10  AverageChargesInCounty        59966 non-null  int64 
 11  AverageCostInFacility         59966 non-null  int64 
 12  AverageChargesInFacility      59966 non-null  int64 
 13  AverageIncomeInZ

In [9]:
data.describe()

,CCSProcedureCode,APRSeverityOfIllnessCode,BirthWeight,AverageCostInCounty,AverageChargesInCounty,AverageCostInFacility,AverageChargesInFacility,AverageIncomeInZipCode,LengthOfStay
count,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000
mean,155.404229,1.254594,3336.298903,2372.806690,7979.126922,2396.414318,7958.472668,59.811143,2.538405
std,89.541978,0.546207,446.244475,639.755096,3220.291347,1248.501189,3859.294711,21.470170,1.171246
min,-1.000000,1.000000,2500.000000,712.000000,1243.000000,457.000000,1120.000000,28.000000,1.000000
25%,115.000000,1.000000,3000.000000,2041.000000,4620.000000,1551.000000,4438.000000,45.000000,2.000000
50%,220.000000,1.000000,3300.000000,2533.000000,9227.000000,1967.000000,7945.000000,55.000000,2.000000
75%,228.000000,1.000000,3600.000000,2785.000000,10644.000000,2895.000000,11619.000000,74.000000,3.000000
max,231.000000,4.000000,7500.000000,3242.000000,11381.000000,8114.000000,18466.000000,115.000000,10.000000


In [15]:
data.describe(include=['object'])

,HealthServiceArea,Gender,Race,TypeOfAdmission,PaymentTypology,EmergencyDepartmentIndicator
count,59966,59966,59966,59966,59966,59966
unique,6,3,4,4,9,2
top,New York City,M,White,Newborn,Medicaid,N
freq,27235,30978,32943,58741,28723,59453


In [21]:
print(data['Gender'].unique())
print(data['HealthServiceArea'].unique())
print(data['Race'].unique())
print(data['TypeOfAdmission'].unique())
print(data['PaymentTypology'].unique())
print(data['EmergencyDepartmentIndicator'].unique())

['F' 'M' 'U']
['New York City' 'Capital/Adirond' 'Central NY' 'Long Island'
 'Southern Tier' 'Western NY']
['Other Race' 'Black/African American' 'White' 'Multi-racial']
['Newborn' 'Emergency' 'Elective' 'Urgent']
['Medicaid' 'Private Health Insurance' 'Blue Cross/Blue Shield' 'Self-Pay'
 'Managed Care, Unspecified' 'Miscellaneous/Other'
 'Federal/State/Local/VA' 'Medicare' 'Unknown']
['N' 'Y']
